<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-46mot1v9/kobert-tokenizer_f32f053ba1ca4ec380c956b0b508a716
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-46mot1v9/kobert-tokenizer_f32f053ba1ca4ec380c956b0b508a716
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [2]:
!pip install transformers

In [3]:
!pip install kobert_tokenizer

In [4]:
!pip install sentencepiece

In [7]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7fc6e015ddb0>)

In [8]:
import pandas as pd

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력


전체 리뷰 개수 : 200000


In [9]:
total_data = total_data.drop(columns = ['ratings'])
total_data = total_data.sample(n=1000, random_state =42)

In [10]:
reviews = total_data['reviews']

In [18]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
import numpy as np

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
model = BertModel.from_pretrained('skt/kobert-base-v1')

pred_result = []

for review in reviews :
  inputs = tokenizer.batch_encode_plus([review])

  with torch.no_grad():
    out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
    logits = out.last_hidden_state
  predicted_class_id = logits.argmax().item()
  pred_result.append(predicted_class_id)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [19]:
result = pd.DataFrame({'reviews' : reviews, 'Pos=0/Neg=1' : pred_result})
result.reset_index(drop=True, inplace=True)
result

,reviews,Pos=0/Neg=1
0,마감이 안좋아요...실밥도 많고 바느질도 부족한 부분이 몇군데 있네요...교환받기 ...,25922
1,깨끗하게 잘 다듬어져 있어요. 맛도좋고요.,1661
2,재구매 배송빨라요 길냥이들이 잘먹어요~~ 대용량이라 좋네요~,6574
3,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,5241
4,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,14254
...,...,...
995,빠배 벌레잘 잡음 단점 대형시켯는데도 크기가 생각보다작음 대형이아니라 중형크기임,17482
996,항상 여기서 사는데 s7엣지는 강화유리가 없어서 아쉽습니다,12711
997,색이 사진과는 다릅니다,4263
998,그냥 그래요 쏘쏘~~ 배송료 넘비싸요,12718


In [ ]:
result.to_csv('result.csv',index = False)